[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/v-artur/Golden_Oreos/blob/main/Modeling.ipynb)

Import files from drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Spectrogram reconstruction 

1.) One person model

Functions for modeling and predicting

In [2]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def create_ae_model(inputsize, outputsize):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(inputsize)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(64, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(32, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(8, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(16, activation="relu"))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(outputsize))
    return model



Testing the optimal structure for one person

In [3]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA


data = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_feat.npy')
spectrogram = np.load(r'/content/drive/MyDrive/DeepLearning/features/sub-01_spec.npy')

#Inital parameters
nfolds = 10
kf = KFold(nfolds,shuffle=False)
pca = PCA()
numComps = 200
explainedVariance = np.zeros(nfolds)


#Initialize an empty spectrogram to save the reconstruction to
rec_spec = np.zeros(spectrogram.shape)
#Save the correlation coefficients for each fold
rs = np.zeros((nfolds,spectrogram.shape[1]))
for k,(train, test) in enumerate(kf.split(data)):
          
    #Normalization
    mu=np.mean(data[train,:],axis=0)
    std=np.std(data[train,:],axis=0)
    trainData=(data[train,:]-mu)/std
    testData=(data[test,:]-mu)/std

    #Fit PCA to training data
    pca.fit(trainData)
    #Get percentage of explained variance by selected components
    explainedVariance[k] = np.sum(pca.explained_variance_ratio_[:numComps])
    #Tranform data into component space
    trainData = np.dot(trainData, pca.components_[:numComps,:].T)
    testData = np.dot(testData, pca.components_[:numComps,:].T)

    #Autoencoder
    early_stopping=EarlyStopping(patience=25, verbose=1, min_delta=1e-5)
    checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=2)

    model = create_ae_model(numComps, spectrogram.shape[1])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model.fit(trainData, spectrogram[train, :], batch_size=64, 
              epochs=500, verbose=2, validation_split=1/9, shuffle=True,
              callbacks=[checkpointer, early_stopping])
            
    #predict with the Autoencoder
    rec_spec[test, :] = model.predict(testData)

    #Evaluate reconstruction of this fold
    for specBin in range(spectrogram.shape[1]):
         #if np.any(np.isnan(rec_spec)):
          #          print('%s has %d broken samples in reconstruction' % (pt, np.sum(np.isnan(rec_spec))))
         r, p = pearsonr(spectrogram[test, specBin], rec_spec[test, specBin])
         rs[k,specBin] = r


#Show evaluation result
print('mean correlation', np.mean(rs))

KeyboardInterrupt: ignored

In [4]:
import os

import numpy as np
import scipy.io.wavfile as wavfile
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA

#import reconstructWave as rW
#import MelFilterBank as mel

# #Optional function
# def createAudio(spectrogram, audiosr=16000, winLength=0.05, frameshift=0.01):
#     mfb = mel.MelFilterBank(int((audiosr*winLength)/2+1), spectrogram.shape[1], audiosr)
#     nfolds = 10
#     hop = int(spectrogram.shape[0]/nfolds)
#     rec_audio = np.array([])
#     for_reconstruction = mfb.fromLogMels(spectrogram)
#     for w in range(0,spectrogram.shape[0],hop):
#         spec = for_reconstruction[w:min(w+hop,for_reconstruction.shape[0]),:]
#         rec = rW.reconstructWavFromSpectrogram(spec,spec.shape[0]*spec.shape[1],fftsize=int(audiosr*winLength),overlap=int(winLength/frameshift))
#         rec_audio = np.append(rec_audio,rec)
#     scaled = np.int16(rec_audio/np.max(np.abs(rec_audio)) * 32767)
#     return scaled


##### MODELING ########

model = True

scores= []

if model == True:
    feat_path = r'/content/drive/MyDrive/DeepLearning/features'
    result_path = r'/content/drive/MyDrive/DeepLearning/features'
    pts = ['sub-%02d'%i for i in range(1,11)]

    winLength = 0.05
    frameshift = 0.01
    audiosr = 16000

    nfolds = 10
    kf = KFold(nfolds,shuffle=False)
    pca = PCA()
    numComps = 200
    
    #Initialize empty matrices for correlation results, randomized controls and amount of explained variance
    allRes = np.zeros((len(pts),nfolds,23))
    explainedVariance = np.zeros((len(pts),nfolds))
    numRands = 1000
    randomControl = np.zeros((len(pts),numRands, 23))

    for pNr, pt in enumerate(pts):
        
        
        #Load the data
        #Dimensions of these data vary depending on the subject
        spectrogram = np.load(os.path.join(feat_path,f'{pt}_spec.npy'))  
        data = np.load(os.path.join(feat_path,f'{pt}_feat.npy'))
        labels = np.load(os.path.join(feat_path,f'{pt}_procWords.npy'))
        featName = np.load(os.path.join(feat_path,f'{pt}_feat_names.npy'))

        

        
        #Initialize an empty spectrogram to save the reconstruction to
        rec_spec = np.zeros(spectrogram.shape)
        #Save the correlation coefficients for each fold
        rs = np.zeros((nfolds,spectrogram.shape[1]))
        for k,(train, test) in enumerate(kf.split(data)):
          
            #Normalization
            mu=np.mean(data[train,:],axis=0)
            std=np.std(data[train,:],axis=0)
            trainData=(data[train,:]-mu)/std
            testData=(data[test,:]-mu)/std

            #Fit PCA to training data
            pca.fit(trainData)
            #Get percentage of explained variance by selected components
            explainedVariance[pNr,k] = np.sum(pca.explained_variance_ratio_[:numComps])
            #Tranform data into component space
            trainData = np.dot(trainData, pca.components_[:numComps,:].T)
            testData = np.dot(testData, pca.components_[:numComps,:].T)

            early_stopping=EarlyStopping(patience=25, verbose=0, min_delta=1e-5)
            checkpointer=ModelCheckpoint(filepath='weights1.hdf5', save_best_only=True, verbose=0)

            #Autoencoder
            model = create_ae_model(numComps, spectrogram.shape[1])
            model.compile(loss='mse', optimizer='adam', metrics=['mse'])
            model.fit(trainData, spectrogram[train, :], batch_size=64, 
                      epochs=500, verbose=0, validation_split=1/9, shuffle=True,
                      callbacks=[checkpointer, early_stopping])
            
            #predict with the Autoencoder
            rec_spec[test, :] = model.predict(testData)

            #Evaluate reconstruction of this fold
            for specBin in range(spectrogram.shape[1]):
                if np.any(np.isnan(rec_spec)):
                    print('%s has %d broken samples in reconstruction' % (pt, np.sum(np.isnan(rec_spec))))
                r, p = pearsonr(spectrogram[test, specBin], rec_spec[test, specBin])
                rs[k,specBin] = r

        #Show evaluation result
        print('%s has mean correlation of %f' % (pt, np.mean(rs)))
        allRes[pNr,:,:]=rs
        scores.append(np.mean(rs))

#         #Estimate random baseline
#         for randRound in range(numRands):
#             #Choose a random splitting point at least 10% of the dataset size away
#             splitPoint = np.random.choice(np.arange(int(spectrogram.shape[0]*0.1),int(spectrogram.shape[0]*0.9)))
#             #Swap the dataset on the splitting point 
#             shuffled = np.concatenate((spectrogram[splitPoint:,:],spectrogram[:splitPoint,:]))
#             #Calculate the correlations
#             for specBin in range(spectrogram.shape[1]):
#                 if np.any(np.isnan(rec_spec)):
#                     print('%s has %d broken samples in reconstruction' % (pt, np.sum(np.isnan(rec_spec))))
#                 r, p = pearsonr(spectrogram[:,specBin], shuffled[:,specBin])
#                 randomControl[pNr, randRound,specBin]=r


#         #Save reconstructed spectrogram
#         os.makedirs(os.path.join(result_path), exist_ok=True)
#         np.save(os.path.join(result_path,f'{pt}_predicted_spec.npy'), rec_spec)
        
#         #Synthesize waveform from spectrogram using Griffin-Lim
#         reconstructedWav = createAudio(rec_spec,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
#         wavfile.write(os.path.join(result_path,f'{pt}_predicted.wav'),int(audiosr),reconstructedWav)

#         #For comparison synthesize the original spectrogram with Griffin-Lim
#         origWav = createAudio(spectrogram,audiosr=audiosr,winLength=winLength,frameshift=frameshift)
#         wavfile.write(os.path.join(result_path,f'{pt}_orig_synthesized.wav'),int(audiosr),origWav)

#     #Save results in numpy arrays          
#     np.save(os.path.join(result_path,'linearResults.npy'),allRes)
#     np.save(os.path.join(result_path,'randomResults.npy'),randomControl)
#     np.save(os.path.join(result_path,'explainedVariance.npy'),explainedVariance)

94/94 [==============================] - 0s 2ms/step
sub-01 has mean correlation of 0.576770
94/94 [==============================] - 0s 2ms/step
sub-02 has mean correlation of 0.592241
94/94 [==============================] - 0s 1ms/step
sub-03 has mean correlation of 0.844132
94/94 [==============================] - 0s 1ms/step
sub-04 has mean correlation of 0.787186
94/94 [==============================] - 0s 1ms/step
sub-05 has mean correlation of 0.556041
94/94 [==============================] - 0s 2ms/step
sub-06 has mean correlation of 0.861184
94/94 [==============================] - 0s 1ms/step
sub-07 has mean correlation of 0.749720
94/94 [==============================] - 0s 2ms/step
sub-08 has mean correlation of 0.658677
94/94 [==============================] - 0s 1ms/step
sub-09 has mean correlation of 0.720659
89/89 [==============================] - 0s 1ms/step
sub-10 has mean correlation of 0.680660


In [5]:
scores

[0.5767700446000605,
 0.5922414102272952,
 0.8441322009617035,
 0.787186120624863,
 0.5560407154650227,
 0.8611840652114083,
 0.7497202821775677,
 0.658677162371156,
 0.7206592165744614,
 0.6806596719513774]